<a href="https://colab.research.google.com/github/lloppy/neironki_5sem/blob/main/Genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager")
}

library(BiocManager)

# Установка и загрузка необходимых пакетов
BiocManager::install("TCGAbiolinks")
library(TCGAbiolinks)

install.packages("magrittr")
library(magrittr)

BiocManager::install("org.Hs.eg.db")
library(org.Hs.eg.db)

install.packages('SummarizedExperiment')
library(SummarizedExperiment)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Installing package(s) 'BiocVersion', 'TCGAbiolinks'

also installing the dependencies ‘plogr’, ‘png’, ‘Biostrings’, ‘RSQLite’, ‘KEGGREST’, ‘filelock’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘matrixStats’, ‘abind’, ‘SparseArray’, ‘AnnotationDbi’, ‘BiocFileCache’, ‘BiocGenerics’, ‘GenomeInfoDb’, ‘XVector’, ‘R.oo’, ‘R.methodsS3’, ‘MatrixGenerics’, ‘Biobase’, ‘S4Arrays’, ‘DelayedArray’, ‘downloader’, ‘biomaRt’, ‘GenomicRanges’, ‘XML’, ‘plyr’, ‘IRanges’, ‘S4Vectors’, ‘R.utils’, ‘SummarizedExperiment’, ‘TCGAbiolinksGUI.data’


Old packages: 'fs', 'xfun'

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOpti

In [ ]:
# Загрузка данных
prad_query <- GDCquery(project = "TCGA-PRAD",
                       data.category = "Transcriptome Profiling",
                       data.type = "Gene Expression Quantification"
)

GDCdownload(prad_query)

# Проверка количества типов образцов
sample_types <- as.vector(as.data.frame(prad_query[[1]][[1]])$sample_type)
table(sample_types)


--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-PRAD

--------------------

oo Filtering results

--------------------

ooo By data.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------


GDCdownload will download 554 files. A total of 2.34840182 GB

The total size of files is big. We will download files in chunks




Downloading: 240 MB     

Downloading: 240 MB     

Downloading: 87 MB     

sample_types
         Metastatic       Primary Tumor Solid Tissue Normal 
                  1                 501                  52 

In [ ]:
# Подготовка данных
prad_data <- GDCprepare(prad_query)
prad_expMat <- assay(prad_data)

# Сохранение матрицы
write.csv(prad_expMat, 'prad_expMat.csv')

# Внимание!!!
# prad_expMat не содержит нормализованных данных!

|====================================================|100%                      Completed after 54 s 


Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

prad subtype information from:doi:10.1016/j.cell.2015.10.025

Available assays in SummarizedExperiment : 
  => unstranded
  => stranded_first
  => stranded_second
  => tpm_unstrand
  => fpkm_unstrand
  => fpkm_uq_unstrand



In [ ]:



# Преобразование имен генов
genes_ensg <- gsub("\\..*", "", rownames(prad_expMat))
genes_symb <- mapIds(org.Hs.eg.db, keys = genes_ensg,
                     keytype = "ENSEMBL", column = "SYMBOL") %>%
  as.vector()

# Набор генов для анализа
set_gen <- c("STAT2", "STAT1", "JAK2", "STAT5A", "JAK1", "SRC", "E2F8", "E2F7", "E2F6", "E2F5",
             "E2F4", "E2F2", "E2F1", "CDKN2A", "CDKN2B", "CDK4", "MYC", "CCND1", "CDKN1B",
             "CCND2", "CDK6", "CDKN1A", "CDC25A", "CDK2", "CCNE1", "STAT3", "STAT5B",
             "E2F3", "CDK1", "CCNB1", "CCNA1", "RBL2", "RBL1", "RB1")

# Определение переменной gene_names для корректного выполнения
gene_names <- genes_symb[genes_symb %in% set_gen]



'select()' returned 1:many mapping between keys and columns



In [ ]:
# Извлечение информации только о генах из нашего набора
filtered_data <- prad_expMat %>%
  as.data.frame() %>%
  .[genes_symb %in% gene_names, ] %>%
  na.omit()

# Вывод результата
filtered_data

# Начинаем анализ!

,TCGA-HC-A6AP-01A-11R-A30B-07,TCGA-KK-A7B2-01A-12R-A32O-07,TCGA-EJ-A8FO-01A-21R-A36G-07,TCGA-G9-6377-01A-11R-1965-07,TCGA-G9-6494-01A-11R-1789-07,TCGA-G9-A9S0-01A-11R-A41O-07,TCGA-KK-A6E5-01A-11R-A311-07,TCGA-J4-A67N-01A-11R-A30B-07,TCGA-VP-A87B-01A-11R-A352-07,TCGA-YL-A9WJ-01A-11R-A37L-07,⋯,TCGA-EJ-A46E-01A-31R-A250-07,TCGA-HC-8265-01A-11R-2263-07,TCGA-V1-A8WS-01A-11R-A37L-07,TCGA-G9-6351-01A-21R-1965-07,TCGA-CH-5768-01A-11R-1580-07,TCGA-J4-AATV-01A-11R-A41O-07,TCGA-HC-8266-01A-11R-2263-07,TCGA-HC-8262-01A-11R-2263-07,TCGA-EJ-7797-01A-11R-2263-07,TCGA-G9-6370-01A-11R-1789-07
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000007968.7,64,51,64,43,36,106,33,47,79,121,⋯,27,88,69,12,144,51,84,79,52,75
ENSG00000080839.12,183,176,274,214,314,201,174,131,149,333,⋯,307,255,470,147,436,235,454,328,290,486
ENSG00000096968.14,460,533,889,374,680,469,553,325,423,1108,⋯,658,529,719,779,970,742,1633,867,849,1091
ENSG00000101412.13,460,184,286,159,279,326,192,333,271,395,⋯,123,277,426,120,291,224,233,402,162,168
ENSG00000103479.16,1189,1277,6381,4394,4716,2081,2589,844,2921,7145,⋯,3861,4552,5852,3526,3795,4305,7308,6233,7066,5843
ENSG00000105173.14,130,61,72,61,94,107,68,41,50,141,⋯,53,34,152,55,86,100,67,109,61,65
ENSG00000105810.10,1644,832,1292,1943,901,990,937,152,6380,1779,⋯,1508,2957,1582,2229,6902,2344,1599,2077,3817,2750
ENSG00000110092.4,6269,4792,10353,8865,8437,5309,7254,4818,8278,9074,⋯,8673,9470,9432,9347,7163,6925,8917,9736,11933,17424
ENSG00000111276.11,2812,2069,4342,3007,3950,1310,2893,1411,3233,2449,⋯,4342,3724,2613,4345,3567,2807,3817,3651,4699,5065


In [ ]:
df <- t(filtered_data)
df

,ENSG00000007968.7,ENSG00000080839.12,ENSG00000096968.14,ENSG00000101412.13,ENSG00000103479.16,ENSG00000105173.14,ENSG00000105810.10,ENSG00000110092.4,ENSG00000111276.11,ENSG00000112242.15,⋯,ENSG00000162434.13,ENSG00000164045.12,ENSG00000165891.16,ENSG00000168610.16,ENSG00000169016.17,ENSG00000170312.16,ENSG00000170581.14,ENSG00000173757.10,ENSG00000197122.12,ENSG00000205250.9
TCGA-HC-A6AP-01A-11R-A30B-07,64,183,460,460,1189,130,1644,6269,2812,311,⋯,2986,350,20,7903,399,577,2721,2820,3129,2663
TCGA-KK-A7B2-01A-12R-A32O-07,51,176,533,184,1277,61,832,4792,2069,275,⋯,3928,120,21,9286,427,120,3303,2802,3795,2498
TCGA-EJ-A8FO-01A-21R-A36G-07,64,274,889,286,6381,72,1292,10353,4342,487,⋯,6006,197,38,19336,663,285,4976,5368,6411,2651
TCGA-G9-6377-01A-11R-1965-07,43,214,374,159,4394,61,1943,8865,3007,816,⋯,5536,255,19,18033,747,154,3677,3376,5067,3277
TCGA-G9-6494-01A-11R-1789-07,36,314,680,279,4716,94,901,8437,3950,571,⋯,7530,514,20,10909,501,184,3637,3669,5115,2186
TCGA-G9-A9S0-01A-11R-A41O-07,106,201,469,326,2081,107,990,5309,1310,725,⋯,2489,152,50,8634,278,689,2367,1652,3831,1991
TCGA-KK-A6E5-01A-11R-A311-07,33,174,553,192,2589,68,937,7254,2893,561,⋯,4653,228,13,10921,548,222,3102,3688,5278,2136
TCGA-J4-A67N-01A-11R-A30B-07,47,131,325,333,844,41,152,4818,1411,301,⋯,1828,93,1,8030,347,96,3820,2231,1758,1671
TCGA-VP-A87B-01A-11R-A352-07,79,149,423,271,2921,50,6380,8278,3233,407,⋯,4302,319,31,6493,414,241,2849,1950,2933,1760
TCGA-YL-A9WJ-01A-11R-A37L-07,121,333,1108,395,7145,141,1779,9074,2449,739,⋯,6694,262,57,15658,546,321,6357,4436,6508,3237


In [ ]:
v1 <- c(0, 1, 2)
v2 <- c(3, 4, 5)

df <- dataframe(col1 = v1, col2 = v2)
df[1:5, ]
df[df$col1 < 1]
df2 <- df[order(df$col1), ]

l1 <- list(df1, df2)

funX <- function(A, B = 10)
{
  Res <- A + B
  return(Res)
}
funX(20)

for(i in 1 : nrow(df)) #число строк в дф, включительно
{
  print(df[1, i])
}
for(i in 1 : nrow(df)) #число строк в дф, включительно
{
  if (df$col1[i] > 10) #while - аналогично
  {
    print(df$col1[i])
  } else if(...)
  {
    ...
  } else
  {

  }
}
?mean()
mean_cl_boot

графика стстистики

In [ ]:
install packages ("ggplot2")
library(ggplot2)

df =
group  value  gender
  A     13      F
  A     27      M
        ...
  D     -2      M

ggplot(df) + aes(group, value) + geom_boxplot() #ящик с усами
geom_point() #точечный граф
geom_bar() #график столбцами
_histogram()
_violin()

#график с условиями
plot1 <- ggplot(df) +
log(value, base = 2) +
aes(group, value, fill = group) +
geom_boxplot(fill = "green/#A935FI", col1 = "grey10" #темнотемно серый
size = 3, width = .5)
theme(legend.position = "none") +
ylab("ValueTest")

#лгкие графики
hist(a1)
plot()
boxplot()

#сохранить файл и указать куда надо
ggsave(plot1, "test.png", dpi = 600, width = 5, height = 3)
setwd("путь")

#открыть файл и сохранить
read("test")
write(a1, "test")
read.csv("test.csv")
write.csv(df, "test_df.csv")

library(openxlsx / xlsx)
read.xlsx( sheet = 2)[,-4]
write.xlsx()

центральная догма молекулярной биологии
днк -> (инф) рнк -> белок
фрнк - функц
трнк - трансп
ирнк=мрнк - инф, матрич - инфа о белках(кодирующий белок protein coding) (на него влияют следующие два)
microRNA miR
long non coding RNA = lncRNA
остальный белки не нужны в целом


In [ ]:
@startuml
actor User as "Зарегистрированный ученик"
rectangle "Система онлайн-обучения" {
    User -> (Выбор курса)
    User -> (Начать курс)
    (Начать курс) -> User: Отображение содержания курса
    User -> (Выбор раздела)
    (Выбор раздела) -> User: Загрузка материалов
    User -> (Изучение материалов)
    User -> (Выполнение задания)
    User -> (Отправить)
    (Отправить) -> User: Отображение результата
    User -> (Переход к следующему разделу)
    (Переход к следующему разделу) -> User: Повторение процесса
    User -> (Завершение курса)
    (Завершение курса) -> User: Получение сертификата
}
@enduml

ERROR: Error in parse(text = input): <text>:1:1: unexpected '@'
1: @
    ^
